# Project 2 - Ames Housing Data and Kaggle Challenge

## Challenge


**The challenge is to predict the sales price of each house. The predicted values for each Id in the test set would be submitted in Kaggle.**


### Contents:
- [Data cleaning of test set](#Data-Import-and-Cleaning)
- [Preparing test set](#Preparing-test-set)
- [Utilise models for prediction](#Utilise-models-for-prediction)

### Data

**Dataset used here:** 
* data_priority_wait_set.csv: this is the training dataset that contains the finalised variables/features. 
* new_test.csv ([source](https://www.kaggle.com/c/dsi-us-11-project-2-regression-challenge/rules)): this dataset contains the test data for the model. Data will be fed into the regression model to make predictions.



*Libraries were added here*

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, Ridge, Lasso

from sklearn import metrics
from sklearn.metrics import mean_squared_error


## Data Import 

In [2]:
train_data = pd.read_csv('../datasets/data_priority_wait_set.csv')
test_data = pd.read_csv('../datasets/new_test.csv')

In [3]:
test_data

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,1662,527377110,60,RL,80.0,8000,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,11,2007,WD
875,1234,535126140,60,RL,90.0,14670,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,MnPrv,NaN,0,8,2008,WD
876,1373,904100040,20,RL,55.0,8250,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2008,WD
877,1672,527425140,20,RL,60.0,9000,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,GdWo,NaN,0,5,2007,WD


In [4]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               879 non-null    int64  
 1   PID              879 non-null    int64  
 2   MS SubClass      879 non-null    int64  
 3   MS Zoning        879 non-null    object 
 4   Lot Frontage     719 non-null    float64
 5   Lot Area         879 non-null    int64  
 6   Street           879 non-null    object 
 7   Alley            58 non-null     object 
 8   Lot Shape        879 non-null    object 
 9   Land Contour     879 non-null    object 
 10  Utilities        879 non-null    object 
 11  Lot Config       879 non-null    object 
 12  Land Slope       879 non-null    object 
 13  Neighborhood     879 non-null    object 
 14  Condition 1      879 non-null    object 
 15  Condition 2      879 non-null    object 
 16  Bldg Type        879 non-null    object 
 17  House Style     

#### Examining missing data

In [5]:
pd.set_option('display.max_rows',85)
test_data.isna().sum().loc[test_data.isna().any()]

Lot Frontage      160
Alley             821
Mas Vnr Type        1
Mas Vnr Area        1
Bsmt Qual          25
Bsmt Cond          25
Bsmt Exposure      25
BsmtFin Type 1     25
BsmtFin Type 2     25
Electrical          1
Fireplace Qu      422
Garage Type        44
Garage Yr Blt      45
Garage Finish      45
Garage Qual        45
Garage Cond        45
Pool QC           875
Fence             707
Misc Feature      838
dtype: int64

### Rename columns

In [6]:
# changing all column names to lower case, and remove special characters and spacing.
test_data.columns = test_data.columns.str.lower()
test_data.columns = test_data.columns.str.replace(' |/','_')
test_data.columns

<ipython-input-6-5f8813c2ba2e>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data.columns = test_data.columns.str.replace(' |/','_')


Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'alley', 'lot_shape', 'land_contour', 'utilities',
       'lot_config', 'land_slope', 'neighborhood', 'condition_1',
       'condition_2', 'bldg_type', 'house_style', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod_add', 'roof_style',
       'roof_matl', 'exterior_1st', 'exterior_2nd', 'mas_vnr_type',
       'mas_vnr_area', 'exter_qual', 'exter_cond', 'foundation', 'bsmt_qual',
       'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1',
       'bsmtfin_type_2', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf',
       'heating', 'heating_qc', 'central_air', 'electrical', '1st_flr_sf',
       '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath',
       'bsmt_half_bath', 'full_bath', 'half_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'kitchen_qual', 'totrms_abvgrd', 'functional',
       'fireplaces', 'fireplace_qu', 'garage_type', 'garage_yr_blt',
       'g

> Follow the steps done in the training set

##### Dealing with continuous data

In [7]:
def var_fill(col,fill):
    '''this functions fills in (fill) into column (col) in test_data'''
    test_data[col] = test_data[col].fillna(fill)
    return test_data[col]

In [8]:
# fill required missing rows in continous variable with 0
# this was previously checked in training set
test_data['lot_frontage'] = var_fill('lot_frontage',0)

##### Dealing with ordinal data

In [9]:
ordinal_columns = ['bsmt_qual','bsmt_cond','bsmt_exposure','bsmtfin_type_1','bsmtfin_type_2','fireplace_qu','garage_finish','garage_qual','garage_cond','pool_qc','fence']

In [10]:
# fill required missing rows in ordinal variables with 'NA'
# this was previously checked in training set
for ord in ordinal_columns:
    test_data[ord] = var_fill(ord,'NA')

In [11]:
# checking that it works
test_data['bsmt_cond'].unique()

array(['TA', 'Gd', 'NA', 'Fa'], dtype=object)

##### Dealing with nominal data

In [12]:
test_data['alley'] = var_fill('alley','NAll')

In [13]:
test_data['garage_type'] = var_fill('garage_type','NG')

In [14]:
test_data['misc_feature'] = var_fill('misc_feature','NMsc')

In [15]:
# checking for any more missing data
test_data.isna().sum().loc[test_data.isna().any()]

mas_vnr_type      1
mas_vnr_area      1
electrical        1
garage_yr_blt    45
dtype: int64

'electrical' is missing one data point, but that is not a variable of interest. 
<br> 'garage_yr_blt' is required to calculate the 'garage_age', but this variables was dropped during modeling.
<br> Both 'mas_vnr_area' and 'mas_vnr_type' are missing one point, and they belong to the variables of interest.


In [16]:
# examining 'mas_vnr_area' and 'mas_vnr_type' variables of interest, still with missing data
test_data['mas_vnr_area'].describe()

count     878.000000
mean      106.982916
std       188.356829
min         0.000000
25%         0.000000
50%         0.000000
75%       173.500000
max      1378.000000
Name: mas_vnr_area, dtype: float64

In [17]:
test_data['mas_vnr_type'].unique()

array(['None', 'BrkFace', 'Stone', 'BrkCmn', 'CBlock', nan], dtype=object)

There is no choice but to replace the missing value in 'mas_vnr_area' as 0. And the missing type in 'mas_vnr_type' would be replaced with 'none'.

In [18]:
test_data['mas_vnr_area'] = var_fill('mas_vnr_area',0)

In [19]:
test_data['mas_vnr_type'] = var_fill('mas_vnr_type','None')

**These are the final 26 (including 4 polynomic) variables, that was selected:**
* overall_qual
* age
* exter_qual
* bsmt_qual
* total_bsmt_sf
* heating_qc
* 1st_flr_sf
* gr_liv_area
* total_bath
* kitchen_qual
* fireplace_qu
* garage_cars
* mas_vnr_area
* lot_area
* bsmtfin_type_1
* overall_qual*gr_liv_area
* exter_qual*1st_flr_sf 
* overall_qual*overall_qual
* total_bsmt_sf*kitchen_qual
* house_style
* exterior_1st
* mas_vnr_type
* foundation
* garage_type
* ms_zoning
* lot_config

### Preparing and processing variables of interest

In [20]:
# converting ordinal variables
r_qual = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
test_data['exter_qual'] = test_data['exter_qual'].map(r_qual)
test_data['bsmt_qual'] = test_data['bsmt_qual'].map(r_qual)
test_data['kitchen_qual'] = test_data['kitchen_qual'].map(r_qual)
test_data['fireplace_qu'] = test_data['fireplace_qu'].map(r_qual)
test_data['heating_qc'] = test_data['heating_qc'].map(r_qual)

r_bsmtfin = {'NA': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6 }
test_data['bsmtfin_type_1'] = test_data['bsmtfin_type_1'].map(r_bsmtfin)

'age' is determined from other variables ['yr_sold','year_built','year_remod_add']

In [21]:
# process age
def find_age(row):
    '''this function calculates and returns the minimum of 2 ages'''
    age_built = row['yr_sold'] - row['year_built']
    age_remod = row['yr_sold'] - row['year_remod_add']
    row['age']=min(age_built,age_remod)
    return row

test_data = test_data.apply(find_age, axis=1)


'total_bath' was determined from other variables ['full_bath','half_bath','bsmt_full_bath','bsmt_half_bath']

In [22]:
# determining total_bath
test_data['total_bath'] = test_data['full_bath'] + (0.5*test_data['half_bath']) + test_data['bsmt_full_bath'] + (0.5*test_data['bsmt_half_bath'])

In [23]:
# These are the four added polynomial features
# Square of 'overall_qual'
test_data['overall_qual*overall_qual'] = test_data['overall_qual'] * test_data['overall_qual']

# Interaction between 'overall_qual' and 'gr_liv_area'
test_data['overall_qual*gr_liv_area'] = test_data['overall_qual'] * test_data['gr_liv_area']

# Interaction between 'exter_qual' and '1st_flr_sf'
test_data['exter_qual*1st_flr_sf'] = test_data['exter_qual'] * test_data['1st_flr_sf']

# Interaction between 'total_bsmt_sf' and 'kitchen_qual'
test_data['total_bsmt_sf*kitchen_qual'] = test_data['total_bsmt_sf'] * test_data['kitchen_qual']


In [24]:
# getting columns from train_data
train_data.columns

Index(['overall_qual', 'age', 'exter_qual', 'bsmt_qual', 'total_bsmt_sf',
       'heating_qc', '1st_flr_sf', 'gr_liv_area', 'total_bath', 'kitchen_qual',
       'fireplace_qu', 'garage_cars', 'house_style', 'exterior_1st',
       'mas_vnr_type', 'foundation', 'garage_type',
       'overall_qual*overall_qual', 'overall_qual*gr_liv_area',
       'exter_qual*1st_flr_sf', 'total_bsmt_sf*kitchen_qual', 'mas_vnr_area',
       'lot_area', 'bsmtfin_type_1', 'ms_zoning', 'lot_config', 'saleprice'],
      dtype='object')

In [25]:
# variables of interest
test = test_data[['overall_qual', 'age', 'exter_qual', 'bsmt_qual', 'total_bsmt_sf','heating_qc', '1st_flr_sf', 'gr_liv_area', 'total_bath', 'kitchen_qual','fireplace_qu', 'garage_cars', 'house_style', 'exterior_1st','mas_vnr_type', 'foundation', 'garage_type','overall_qual*overall_qual', 'exter_qual*1st_flr_sf', 'mas_vnr_area','lot_area', 'bsmtfin_type_1', 'ms_zoning', 'lot_config']]

In [26]:
# double checking for missing data
pd.set_option('display.max_rows',30)
test.isnull().sum()

overall_qual                 0
age                          0
exter_qual                   0
bsmt_qual                    0
total_bsmt_sf                0
heating_qc                   0
1st_flr_sf                   0
gr_liv_area                  0
total_bath                   0
kitchen_qual                 0
fireplace_qu                 0
garage_cars                  0
house_style                  0
exterior_1st                 0
mas_vnr_type                 0
foundation                   0
garage_type                  0
overall_qual*overall_qual    0
exter_qual*1st_flr_sf        0
mas_vnr_area                 0
lot_area                     0
bsmtfin_type_1               0
ms_zoning                    0
lot_config                   0
dtype: int64

In [27]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   overall_qual               879 non-null    int64  
 1   age                        879 non-null    int64  
 2   exter_qual                 879 non-null    int64  
 3   bsmt_qual                  879 non-null    int64  
 4   total_bsmt_sf              879 non-null    int64  
 5   heating_qc                 879 non-null    int64  
 6   1st_flr_sf                 879 non-null    int64  
 7   gr_liv_area                879 non-null    int64  
 8   total_bath                 879 non-null    float64
 9   kitchen_qual               879 non-null    int64  
 10  fireplace_qu               879 non-null    int64  
 11  garage_cars                879 non-null    int64  
 12  house_style                879 non-null    object 
 13  exterior_1st               879 non-null    object 

### Dealing with training set (train_data)

In [28]:
# note include saleprice
train_data.head()
train_data.drop(columns=['overall_qual*gr_liv_area','total_bsmt_sf*kitchen_qual'],inplace=True)

In [29]:
y_train = train_data['saleprice']

#### One-hot encoding the nominal variables

In [30]:
# select columns with nominal variables
nominal_train = train_data.select_dtypes(include=['object']).copy()

In [31]:
# examining the number of types in each nominal variables
for col in nominal_train.columns:
    print(col, ':', len(nominal_train[col].unique()))

house_style : 8
exterior_1st : 14
mas_vnr_type : 4
foundation : 6
garage_type : 6
ms_zoning : 7
lot_config : 5


In [32]:
dummy_train = pd.get_dummies(data=nominal_train, columns=nominal_train.columns, drop_first=True)

dummy_train.reset_index(inplace=True)
dummy_train.head()

,index,house_style_1.5Unf,house_style_1Story,house_style_2.5Fin,house_style_2.5Unf,house_style_2Story,house_style_SFoyer,house_style_SLvl,exterior_1st_BrkComm,exterior_1st_BrkFace,...,ms_zoning_C (all),ms_zoning_FV,ms_zoning_I (all),ms_zoning_RH,ms_zoning_RL,ms_zoning_RM,lot_config_CulDSac,lot_config_FR2,lot_config_FR3,lot_config_Inside
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,3,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


##### Scaling the numerical data


In [33]:
# once again, select columns with numerical variables
numerical_train = train_data.select_dtypes(include=['int64','float64']).copy() 
numerical_train.drop(columns=['saleprice'],inplace=True)

# 15+4 numerical variables
numerical_train.head()

,overall_qual,age,exter_qual,bsmt_qual,total_bsmt_sf,heating_qc,1st_flr_sf,gr_liv_area,total_bath,kitchen_qual,fireplace_qu,garage_cars,overall_qual*overall_qual,exter_qual*1st_flr_sf,mas_vnr_area,lot_area,bsmtfin_type_1
0,6,5,4,3,725.0,5,725,1479,2.5,4,0,2.0,36,2900,289.0,13517,6
1,7,12,4,4,913.0,5,913,2122,3.5,4,3,2.0,49,3652,132.0,11492,6
2,5,3,3,3,1057.0,3,1057,1057,2.0,4,0,1.0,25,3171,0.0,7922,6
3,5,3,3,4,384.0,4,744,1444,2.5,3,0,2.0,25,2232,0.0,9802,1
4,6,17,3,2,676.0,3,831,1445,2.0,3,0,2.0,36,2493,0.0,14235,1


In [34]:
# instantiate sc
sc = StandardScaler()

# fit and transform only training data
X_scaled_train = pd.DataFrame(sc.fit_transform(numerical_train))
X_scaled_train.columns = numerical_train.columns
X_scaled_train.reset_index(inplace=True)
X_scaled_train.head()

,index,overall_qual,age,exter_qual,bsmt_qual,total_bsmt_sf,heating_qc,1st_flr_sf,gr_liv_area,total_bath,kitchen_qual,fireplace_qu,garage_cars,overall_qual*overall_qual,exter_qual*1st_flr_sf,mas_vnr_area,lot_area,bsmtfin_type_1
0,0,-0.131476,-0.874814,0.984330,-0.593464,-0.810482,0.859494,-1.195349,-0.061439,0.331925,0.702381,-1.020794,0.189303,-0.234186,-0.663046,1.053581,0.505040,1.124705
1,1,0.588815,-0.537441,0.984330,0.550390,-0.365648,0.859494,-0.692310,1.281421,1.594889,0.702381,0.641220,0.189303,0.499204,-0.247930,0.160731,0.202715,1.124705
2,2,-0.851766,-0.971206,-0.726614,-0.593464,-0.024925,-1.240637,-0.307004,-0.942757,-0.299558,0.702381,-1.020794,-1.344372,-0.854746,-0.513450,-0.589946,-0.330273,1.124705
3,3,-0.851766,-0.971206,-0.726614,0.550390,-1.617334,-0.190571,-1.144510,-0.134534,0.331925,-0.823701,-1.020794,0.189303,-0.854746,-1.031793,-0.589946,-0.049596,-1.253570
4,4,-0.131476,-0.296461,-0.726614,-1.737319,-0.926422,-1.240637,-0.911721,-0.132446,-0.299558,-0.823701,-1.020794,0.189303,-0.234186,-0.887716,-0.589946,0.612234,-1.253570


##### Recombine both nominal and numerical dataframe

In [35]:
# concating both dataframes
X_train = pd.concat([X_scaled_train,dummy_train], axis=1)
X_train.drop(columns=['index'],inplace=True)
X_train.head()

,overall_qual,age,exter_qual,bsmt_qual,total_bsmt_sf,heating_qc,1st_flr_sf,gr_liv_area,total_bath,kitchen_qual,...,ms_zoning_C (all),ms_zoning_FV,ms_zoning_I (all),ms_zoning_RH,ms_zoning_RL,ms_zoning_RM,lot_config_CulDSac,lot_config_FR2,lot_config_FR3,lot_config_Inside
0,-0.131476,-0.874814,0.984330,-0.593464,-0.810482,0.859494,-1.195349,-0.061439,0.331925,0.702381,...,0,0,0,0,1,0,1,0,0,0
1,0.588815,-0.537441,0.984330,0.550390,-0.365648,0.859494,-0.692310,1.281421,1.594889,0.702381,...,0,0,0,0,1,0,1,0,0,0
2,-0.851766,-0.971206,-0.726614,-0.593464,-0.024925,-1.240637,-0.307004,-0.942757,-0.299558,0.702381,...,0,0,0,0,1,0,0,0,0,1
3,-0.851766,-0.971206,-0.726614,0.550390,-1.617334,-0.190571,-1.144510,-0.134534,0.331925,-0.823701,...,0,0,0,0,1,0,0,0,0,1
4,-0.131476,-0.296461,-0.726614,-1.737319,-0.926422,-1.240637,-0.911721,-0.132446,-0.299558,-0.823701,...,0,0,0,0,1,0,0,0,0,1


In [36]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1912 entries, 0 to 1911
Data columns (total 60 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   overall_qual               1912 non-null   float64
 1   age                        1912 non-null   float64
 2   exter_qual                 1912 non-null   float64
 3   bsmt_qual                  1912 non-null   float64
 4   total_bsmt_sf              1912 non-null   float64
 5   heating_qc                 1912 non-null   float64
 6   1st_flr_sf                 1912 non-null   float64
 7   gr_liv_area                1912 non-null   float64
 8   total_bath                 1912 non-null   float64
 9   kitchen_qual               1912 non-null   float64
 10  fireplace_qu               1912 non-null   float64
 11  garage_cars                1912 non-null   float64
 12  overall_qual*overall_qual  1912 non-null   float64
 13  exter_qual*1st_flr_sf      1912 non-null   float

---

### Dealing with test set

#### One-hot encoding the nominal variables

In [37]:
# select columns with nominal variables
nominal_test = test.select_dtypes(include=['object']).copy()

# examining the number of types in each nominal variables
for col in nominal_test.columns:
    print(col, ':', len(nominal_test[col].unique()))

house_style : 8
exterior_1st : 13
mas_vnr_type : 5
foundation : 6
garage_type : 7
ms_zoning : 6
lot_config : 5


One should note showed that all categories might not be in all nominal variables/features, which will pose a problem later, namely, 'exterior_1st' and 'ms_zoning'. 

In [38]:
dummy_test = pd.get_dummies(data=nominal_test, columns=nominal_test.columns, drop_first=True)
dummy_test.reset_index(inplace=True)

dummy_test.shape

(879, 44)

#### Checking for same number of columns in both dummy (train and test) sets


In [39]:
# Get missing columns in the training test
missing_cols = set(dummy_train.columns) - set(dummy_test.columns)

# Add a missing column in test set with default value equal to 0
for col in missing_cols:
    dummy_test[col] = 0
    
# Ensure the order of column in the test set is in the same order than in train set
dummy_test = dummy_test[dummy_train.columns]

dummy_test.shape

(879, 44)

##### Scaling the numerical data


In [40]:
# once again, select columns with numerical variables
numerical_test = test.select_dtypes(include=['int64','float64']).copy() 
numerical_test.head()

,overall_qual,age,exter_qual,bsmt_qual,total_bsmt_sf,heating_qc,1st_flr_sf,gr_liv_area,total_bath,kitchen_qual,fireplace_qu,garage_cars,overall_qual*overall_qual,exter_qual*1st_flr_sf,mas_vnr_area,lot_area,bsmtfin_type_1
0,6,56,3,2,1020,4,908,1928,2.0,2,0,1,36,2724,0.0,9142,1
1,5,29,3,4,1967,3,1967,1967,2.0,3,0,2,25,5901,0.0,9662,1
2,7,0,4,4,654,5,664,1496,3.5,4,4,2,49,2656,0.0,17104,6
3,5,1,4,3,968,3,968,968,1.0,3,0,2,25,3872,0.0,8520,1
4,6,46,3,4,1394,4,1394,1394,2.5,3,4,2,36,4182,247.0,9500,4


In [41]:
numerical_train.head()

,overall_qual,age,exter_qual,bsmt_qual,total_bsmt_sf,heating_qc,1st_flr_sf,gr_liv_area,total_bath,kitchen_qual,fireplace_qu,garage_cars,overall_qual*overall_qual,exter_qual*1st_flr_sf,mas_vnr_area,lot_area,bsmtfin_type_1
0,6,5,4,3,725.0,5,725,1479,2.5,4,0,2.0,36,2900,289.0,13517,6
1,7,12,4,4,913.0,5,913,2122,3.5,4,3,2.0,49,3652,132.0,11492,6
2,5,3,3,3,1057.0,3,1057,1057,2.0,4,0,1.0,25,3171,0.0,7922,6
3,5,3,3,4,384.0,4,744,1444,2.5,3,0,2.0,25,2232,0.0,9802,1
4,6,17,3,2,676.0,3,831,1445,2.0,3,0,2.0,36,2493,0.0,14235,1


In [42]:
numerical_test.shape

(879, 17)

In [43]:
# sc = StandardScaler()

# only transform on test set with sc
X_scaled_test = pd.DataFrame(sc.transform(numerical_test))
X_scaled_test.columns = numerical_test.columns
X_scaled_test.reset_index(inplace=True)

X_scaled_test.head()

,index,overall_qual,age,exter_qual,bsmt_qual,total_bsmt_sf,heating_qc,1st_flr_sf,gr_liv_area,total_bath,kitchen_qual,fireplace_qu,garage_cars,overall_qual*overall_qual,exter_qual*1st_flr_sf,mas_vnr_area,lot_area,bsmtfin_type_1
0,0,-0.131476,1.583185,-0.726614,-1.737319,-0.112472,-0.190571,-0.705689,0.876266,-0.299558,-2.349783,-1.020794,-1.344372,-0.234186,-0.760201,-0.589946,-0.148132,-1.253570
1,1,-0.851766,0.281891,-0.726614,0.550390,2.128258,-1.240637,2.127916,0.957714,-0.299558,-0.823701,-1.020794,0.189303,-0.854746,0.993553,-0.589946,-0.070498,-1.253570
2,2,0.588815,-1.115794,0.984330,0.550390,-0.978477,0.859494,-1.358569,-0.025936,1.594889,0.702381,1.195224,0.189303,0.499204,-0.797738,-0.589946,1.040566,1.124705
3,3,-0.851766,-1.067598,0.984330,-0.593464,-0.235511,-1.240637,-0.545145,-1.128627,-1.562523,-0.823701,-1.020794,0.189303,-0.854746,-0.126487,-0.589946,-0.240994,-1.253570
4,4,-0.131476,1.101224,-0.726614,0.550390,0.772463,-0.190571,0.594719,-0.238956,0.331925,-0.823701,1.195224,0.189303,-0.234186,0.044638,0.814729,-0.094684,0.173395


##### Recombine both nominal and numerical dataframe


In [44]:
# concating both dataframes
X_test = pd.concat([X_scaled_test,dummy_test], axis=1)
X_test.drop(columns=['index'],inplace=True)

pd.set_option('display.max_columns',50)
X_test.head()

,overall_qual,age,exter_qual,bsmt_qual,total_bsmt_sf,heating_qc,1st_flr_sf,gr_liv_area,total_bath,kitchen_qual,fireplace_qu,garage_cars,overall_qual*overall_qual,exter_qual*1st_flr_sf,mas_vnr_area,lot_area,bsmtfin_type_1,house_style_1.5Unf,house_style_1Story,house_style_2.5Fin,house_style_2.5Unf,house_style_2Story,house_style_SFoyer,house_style_SLvl,exterior_1st_BrkComm,...,exterior_1st_Wd Sdng,exterior_1st_WdShing,mas_vnr_type_BrkFace,mas_vnr_type_None,mas_vnr_type_Stone,foundation_CBlock,foundation_PConc,foundation_Slab,foundation_Stone,foundation_Wood,garage_type_Attchd,garage_type_Basment,garage_type_BuiltIn,garage_type_CarPort,garage_type_Detchd,ms_zoning_C (all),ms_zoning_FV,ms_zoning_I (all),ms_zoning_RH,ms_zoning_RL,ms_zoning_RM,lot_config_CulDSac,lot_config_FR2,lot_config_FR3,lot_config_Inside
0,-0.131476,1.583185,-0.726614,-1.737319,-0.112472,-0.190571,-0.705689,0.876266,-0.299558,-2.349783,-1.020794,-1.344372,-0.234186,-0.760201,-0.589946,-0.148132,-1.253570,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
1,-0.851766,0.281891,-0.726614,0.550390,2.128258,-1.240637,2.127916,0.957714,-0.299558,-0.823701,-1.020794,0.189303,-0.854746,0.993553,-0.589946,-0.070498,-1.253570,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.588815,-1.115794,0.984330,0.550390,-0.978477,0.859494,-1.358569,-0.025936,1.594889,0.702381,1.195224,0.189303,0.499204,-0.797738,-0.589946,1.040566,1.124705,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,-0.851766,-1.067598,0.984330,-0.593464,-0.235511,-1.240637,-0.545145,-1.128627,-1.562523,-0.823701,-1.020794,0.189303,-0.854746,-0.126487,-0.589946,-0.240994,-1.253570,0,1,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
4,-0.131476,1.101224,-0.726614,0.550390,0.772463,-0.190571,0.594719,-0.238956,0.331925,-0.823701,1.195224,0.189303,-0.234186,0.044638,0.814729,-0.094684,0.173395,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [45]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 60 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   overall_qual               879 non-null    float64
 1   age                        879 non-null    float64
 2   exter_qual                 879 non-null    float64
 3   bsmt_qual                  879 non-null    float64
 4   total_bsmt_sf              879 non-null    float64
 5   heating_qc                 879 non-null    float64
 6   1st_flr_sf                 879 non-null    float64
 7   gr_liv_area                879 non-null    float64
 8   total_bath                 879 non-null    float64
 9   kitchen_qual               879 non-null    float64
 10  fireplace_qu               879 non-null    float64
 11  garage_cars                879 non-null    float64
 12  overall_qual*overall_qual  879 non-null    float64
 13  exter_qual*1st_flr_sf      879 non-null    float64

### Utilise models for prediction

#### *linear regression (lr_total) model with 26 variables (inc 4 polynomial features)*

In [46]:
# test on finalised features from preprocessing notebook
# training data is X_train, y_train
# test set is X_test

# instantiate
lr_total = LinearRegression()

# fit
lr_total.fit(X_train, y_train)

# predict
y_pred = lr_total.predict(X_test)

# rsme scores on training set
print(f'The rmsme for the training set is: {np.sqrt(metrics.mean_squared_error(y_train, lr_total.predict(X_train)))}')

# y
kaggle_26 = test_data[['id']]
kaggle_26['saleprice'] = pd.DataFrame(y_pred)

The rmsme for the training set is: 24205.713404501617


<ipython-input-46-153b226e9e67>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle_26['saleprice'] = pd.DataFrame(y_pred)


In [47]:
kaggle_26

,id,saleprice
0,2658,149002.486934
1,2718,169386.492584
2,2414,202608.244140
3,1989,101426.771572
4,625,167160.023184
...,...,...
874,1662,188614.513689
875,1234,224008.951516
876,1373,130406.720586
877,1672,101970.882435


**Save predicted results as csv file for kaggle submission.**

In [48]:
kaggle_26.to_csv('../datasets/kaggle_26.csv',index=False)

#### *Ridge regression (ridge_model) model with 26 variables (inc 4 polynomial features)*

In [49]:
# test on finalised features from preprocessing notebook
# training data is X_train, y_train
# test set is X_test

# Instantiate.
# Decided to use default value since there was little difference between 1.718 and 1.0 previously
ridge_model = Ridge(alpha=1.0)

# Fit model using best ridge alpha!
ridge_model.fit(X_train, y_train)

# predict
y_pred_r = ridge_model.predict(X_test)

# rsme scores on training set
print(f'The rmsme for the training set is: {np.sqrt(mean_squared_error(y_train, ridge_model.predict(X_train)))}')

# y
kaggle_r26 = test_data[['id']]
kaggle_r26['saleprice'] = pd.DataFrame(y_pred_r)


The rmsme for the training set is: 24261.76385677918


<ipython-input-49-51a7dc3feac3>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle_r26['saleprice'] = pd.DataFrame(y_pred_r)


In [50]:
kaggle_r26

,id,saleprice
0,2658,145116.406355
1,2718,170042.722632
2,2414,203412.095675
3,1989,101884.069918
4,625,167615.992211
...,...,...
874,1662,188268.254520
875,1234,224490.491719
876,1373,130353.268904
877,1672,100935.814832


**Save predicted results as csv file for kaggle submission.**

In [51]:
kaggle_r26.to_csv('../datasets/kaggle_r26.csv',index=False)

#### *Lasso regression (lasso_model) model with 26 variables (inc 4 polynomial features)*

In [52]:
# test on finalised features from preprocessing notebook
# training data is X_train, y_train
# test set is X_test

# using default value
# Instantiate.
lasso_model = Lasso(alpha=1.0)

# Fit model using best ridge alpha!
lasso_model.fit(X_train, y_train)

# predict
y_pred_l = lasso_model.predict(X_test)

# rsme scores on training set
print(f'The rmsme for the training set is: {np.sqrt(mean_squared_error(y_train, lasso_model.predict(X_train)))}')

# y
kaggle_l26 = test_data[['id']]
kaggle_l26['saleprice'] = pd.DataFrame(y_pred_l)


The rmsme for the training set is: 24207.09555054204


C:\Users\USER\anaconda3\envs\dsi24\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15172415029.232422, tolerance: 1187352356.975226
  model = cd_fast.enet_coordinate_descent(
<ipython-input-52-363455f0f9a6>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle_l26['saleprice'] = pd.DataFrame(y_pred_l)


In [53]:
kaggle_l26

,id,saleprice
0,2658,148742.358532
1,2718,169266.168335
2,2414,202630.162502
3,1989,101380.395553
4,625,167171.292311
...,...,...
874,1662,188585.120986
875,1234,224032.074219
876,1373,130374.380945
877,1672,101889.249384


**Save predicted results as csv file for kaggle submission.**

In [54]:
kaggle_l26.to_csv('../datasets/kaggle_l26.csv',index=False)

### Results table

| model | description | hyperparams | nr features | train RSME | kaggle RSME
|---|---|---------|-------|-----------|-----------|
|lr| linear regression | - | 26 | 24205.71 | 27858.99
|ridge| ridge | alpha=1.0 | 26| 24261.76 | 27844.41
|lasso| lasso | alpha=1.0 | 26 | 24207.10 | 28059.36